# 9.1题求解过程

In [10]:
import chapter9module as c9m

In [11]:
a = 2
b = 0.6
t = 0.012
q = 6.5 / 0.01 / 0.01
sigma_0 = 18.8e6

## 求解（a）：
1. 计算 A 点弯矩和应力

In [12]:
M_A = c9m.bendingMomentFixedEnd1(q, b)
print(f"A点弯矩 M_A 为 {M_A :.2f} N*m")
sigma_A = c9m.bendingNormalStress(M_A, t)
print(f"A点由弯曲引起的正应力 sigma_A 为 {sigma_A*1e-6 :.2f} MPa")

A点弯矩 M_A 为 975.00 N*m
A点由弯曲引起的正应力 sigma_A 为 40.62 MPa


2. 计算 B 点弯矩和应力

In [13]:
M_B = c9m.bendingMomentFixedEnd2(q, b)
print(f"B点弯矩 M_B 为 {M_B :.2f} N*m")
sigma_B = c9m.bendingNormalStress(M_B, t)
print(f"B点由弯曲引起的正应力 sigma_B 为 {sigma_B*1e-6 :.2f} MPa")

B点弯矩 M_B 为 1950.00 N*m
B点由弯曲引起的正应力 sigma_B 为 81.25 MPa


3. 计算 A 点挠度

In [14]:
w_A = c9m.midSpanDeflection1(q, b, c9m.E1(), t)
print(f"A点挠度 w_A 为 {w_A*1e3 :.3f} mm")

A点挠度 w_A 为 0.693 mm


## 求解（b）：
1. 计算板条梁中面拉力

In [15]:
T = 18.8e6 * 1 * t
print(f"板条梁中面拉力 T 为 {T :.2f} N")

板条梁中面拉力 T 为 225600.00 N


2. 计算系数 u

In [16]:
u = c9m.u(b, T, c9m.cylindricalStiffness(t))
print(f"u = {u :.3f}")

u = 0.782


3. 计算 A 点辅助函数、弯矩、正应力

In [17]:
print(f"phai1(u) = {c9m.phai1(u) :.3f}")
M_A_complex = M_A * c9m.phai1(u)
print(f"A点由复杂弯曲引起的弯矩 M_A_complex 为 {M_A_complex :.2f} N*m")
sigma_A_complex = c9m.bendingNormalStress(M_A_complex, t)
print(f"A点由复杂弯曲引起的正应力 sigma_A_complex 为 {(sigma_0+sigma_A_complex)*1e-6 :.2f} MPa")
print(f"f1(u) = {c9m.f1(u) :.3f}")
w_A_complex = w_A * c9m.f1(u)
print(f"A点复杂弯曲挠度 w_A_complex 为 {w_A_complex*1e3 :.3f} mm")

phai1(u) = 0.933
A点由复杂弯曲引起的弯矩 M_A_complex 为 909.71 N*m
A点由复杂弯曲引起的正应力 sigma_A_complex 为 56.70 MPa
f1(u) = 0.942
A点复杂弯曲挠度 w_A_complex 为 0.653 mm


4. 计算 B 点弯矩和应力

In [18]:
print(f"X(u) = {c9m.X(u) :.3f}")
M_B_complex = c9m.bendingMomentFixedEnd2(q, b) * c9m.X(u)
print(f"B点由复杂弯曲引起的弯矩 M_B_complex 为 {M_B_complex :.2f} N*m")
sigma_B_complex = c9m.bendingNormalStress(M_B_complex, t)
print(f"B点由复杂弯曲引起的正应力 sigma_B_complex 为 {(sigma_0+sigma_B_complex)*1e-6 :.2f} MPa")

X(u) = 0.962
B点由复杂弯曲引起的弯矩 M_B_complex 为 1874.93 N*m
B点由复杂弯曲引起的正应力 sigma_B_complex 为 96.92 MPa
